# Plot parameters obtained in CIGALE in the trained SOM

## Define the directory
Define the directory were all the files are saved.

In [1]:
observatory = '/data/mfonseca/'
my_computer = '/home/polaris/Lab_Astro/data/'

directory = my_computer

## Import the libraries

In [2]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import numpy as np
import pandas as pd

# SimpSOM, https://simpsom.readthedocs.io
import simpsom as sps

# Astropy
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u

# CuPY
# import cupy as cp

# Seaborn
import seaborn as sns

## Trained SOM

Import original and whitened data used to train the SOM

In [22]:
data_path = '/home/polaris/Lab_Astro/data/soms/maps1/EMU_0102-32_DESY6GOLD_VIKINGDR5_CATWISE_noMag_original.csv'
data = pd.read_csv(data_path, sep=',', header=0, index_col=0)

data_white_path = '/home/polaris/Lab_Astro/data/soms/maps1/EMU_0102-32_DESY6GOLD_VIKINGDR5_CATWISE_noMag_zcaWhite.csv'
data_white = pd.read_csv(data_white_path, sep=',', header=0, index_col=0)

Import the trained SOM

In [ ]:
best_som_epoch = 40
file_trained_som = directory + 'soms/maps3/epoch_files/trained_som_' + str(best_som_epoch) + 'epoch'+ '.npy'

In [ ]:
sommap = sps.SOMNet(
    17,
    16,
    data_white,
    load_file = file_trained_som,
    debug= False,
    metric = 'euclidean',
    topology= 'hexagonal'
    )

## CIGALE results

Import the parameters from CIGALE

In [3]:
cigale_results_file_path = directory + 'cigale/all_data_6/out7/results.fits'
cigale_results_table = Table.read(cigale_results_file_path)
cigale_results_df = cigale_results_table.to_pandas()

In [12]:
# Lowest chi-square
min_index = cigale_results_df['best.reduced_chi_square'].idxmin()
min_id = cigale_results_df.loc[min_index, 'id']

# Highest chi-square
max_index = cigale_results_df['best.reduced_chi_square'].idxmax()
max_id = cigale_results_df.loc[max_index, 'id']

print("ID with lowest chi-square:", min_id)
print("ID with highest chi-square:", max_id) 

# Compute the median value
median_value = cigale_results_df['best.reduced_chi_square'].median()

# Find the row where chi-square is closest to the median
# (in case there's no exact match)
closest_index = (cigale_results_df['best.reduced_chi_square'] - median_value).abs().idxmin()
median_id = cigale_results_df.loc[closest_index, 'id']

print("ID with chi-square closest to the median:", median_id)

ID with lowest chi-square: b'J010240-312004'
ID with highest chi-square: b'J004953-321718'
ID with chi-square closest to the median: b'J005649-332450'


## Join both tables

In [ ]:
allresults_df = pd.merge(data_white, cigale_results_df, on='key')

## Plot the results in the trained SOM